# Metrics evaluation notebook

_Alex Malz (NYU)_
_Renee Hlozek (Toronto)_
_Rahul Biswas (Stockholm University)_
_Tarek Alam (UCL)_
_Rafael Martinez-Galarza (Harvard)_
_Anita Bahmanyar (Toronto)_


This notebook defines the metrics (and corresponding truth tables) to be run via an input file and to produce plots based on the output.

In [ ]:
import string
import itertools
import random
import numpy as np
import sklearn as skl
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
%matplotlib inline
import proclam
from proclam import *

## Define the inputs

In [ ]:
metricslist = ['Brier']
dirname = 'classification_probabilities/'
classifications = ['RandomForest/predicted_prob_RandomForest.csv']
truth_tables = ['RandomForest/truth_table_RandomForest.csv']
class_pairs = zip(classifications, truth_tables)


In [ ]:
def read_class_pairs(pair):
    import pandas as pd
    clfile = pair[0]
    truthfile = pair[1]

    prob_mat = pd.read_csv(dirname+clfile, delim_whitespace=True).values
    nobj = np.shape(prob_mat)[0]
    nclass = np.shape(prob_mat)[1]
    truth_values = pd.read_csv(dirname+truthfile, delim_whitespace=True).values
    nobj_truth = np.shape(truth_values)[0]
    nclass_truth = np.shape(truth_values)[1]
    tvec = np.where(truth_values==1)[1]
    if nclass_truth!= nclass:
        print('Truth table of size %i x %i and prob matrix of size %i x %i do not match up in size'%(nobj,nclass,nobj_truth,nclass_truth))
    else:
        print('Considering classifications with %i classes'%nclass)
    pmat = prob_mat
    return pmat, tvec

In [ ]:
for pair in class_pairs:
    probm, truthv = read_class_pairs(pair)
    for metric in metricslist:
        print('Using metric %s'%metric)
        D = getattr(proclam.metrics, metric)()
        hm = D.evaluate(probm,truthv)
        print(hm)
        
        
        pl.plot()
    #print(cm, tm)
#from proclam.simulators import simulator
#from proclam.simulators import logunbalanced

In [ ]:
# make a plot of classifier on the x-axis and metric score on the y-axis
